# Setups

In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdrive')

os.chdir(os.path.join(os.getcwd(), 'gdrive', 'Shareddrives', 'anzi'))

Mounted at /content/gdrive


In [ ]:
!pip install tensorflow-addons
!pip install transformers
!pip install datasets

In [ ]:
import torch
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from nltk.tokenize import word_tokenize

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert = TFBertModel.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


# Data preprocessing
#### 0: O
#### 1: B
#### 2: I
#### 3: E
#### 4: S


In [ ]:
import json
import re
import pandas as pd

In [ ]:
with open("./CLOTH/CLOTH_train_cleaned.json", "r") as file:
    training_dataset = json.load(file)

In [ ]:
with open("./S/data.json", "r") as file:
    translated_dataset = json.load(file)
translated = []
year = list(translated_dataset['question'].keys())
for y in year:
    for index in range(len(translated_dataset['question'][y])):
        translated.append({'sentence': translated_dataset['question'][y][index], 'answer': translated_dataset['answer'][y][index]})

In [ ]:
GSAT = []
GSAT_dataset = pd.read_excel('./J/new_compare.xlsx', sheet_name=None)
for G in GSAT_dataset.values():
    for index, row in G.iterrows():
        answer_choice = row['Answer'].replace(' ', '')
        GSAT.append({'sentence': row['Sentence'], 'answer': row[answer_choice].replace(' ', '')})

In [ ]:
len((translated + GSAT * 5) * 20)

83040

In [ ]:
from tqdm import tqdm

inputs = {'input_ids': [], 'attention_mask': []}
targets = []
count = 0
for data in tqdm(training_dataset + (translated + GSAT * 5) * 20):
    mask_sentence = data["sentence"]
    answer = data["answer"]

    sentence = mask_sentence.replace('[MASK]', answer)
    tokenized = tokenizer(sentence, truncation=True, padding='max_length', max_length=64, return_tensors='tf')
    tokenized_answer = tokenizer(answer, return_tensors='tf').input_ids[0][1:-1]

    target = np.zeros(shape=(64))

    try:
        if len(tokenized_answer) == 1:
            target[np.where(tokenized.input_ids[0] == tokenized_answer)[0][0]] = 4
        else:
            index = np.where(tokenized.input_ids[0] == tokenized_answer[0])[0][0]
            target[index] = 1
            for i in range(1, len(tokenized_answer) - 1):
                target[index+i] = 2
            target[index + len(tokenized_answer) - 1] = 3
        targets.append(target)

        inputs['input_ids'].append(tokenized.input_ids[0])
        inputs['attention_mask'].append(tokenized.attention_mask[0])
    except Exception as err:
        count += 1
inputs = {k: np.array(v) for k, v in inputs.items()}
targets = np.array(targets)

100%|██████████| 159890/159890 [10:09<00:00, 262.46it/s]


# BERT

In [ ]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
class predictCloze:
    def __init__(self, tokenizer, bert):
        x = Input(shape=(768))
        h = Dense(512)(x)
        h = Dense(128)(x)
        h = Dense(32)(x)
        y = Dense(1, activation='sigmoid')(x)
        self.model = Model(inputs=x, outputs=y)
        self.model.compile(optimizer=Adam(1e-3), loss=BinaryCrossentropy())

        self.tokenizer = tokenizer
        self.bert = bert

    def trainPreprocess(self, training):
        inputs = []
        targets = []
        for data in training:
            sentence = data['sentence']
            answer = data['answer']

            if len(self.tokenizer(answer, return_tensors='tf').input_ids[0]) == 3:
                input = self.tokenizer(sentence.replace('[MASK]', answer), return_tensors='tf')
                target = [int(token == 103) for token in self.tokenizer(sentence, return_tensors='tf').input_ids[0][1:-1]]

                inputs.append(input)
                target.append(np.array(target))

        return inputs, targets

    def testPreprocess(self, testing):
        inputs = []
        for data in testing:
            inputs.append(self.tokenizer(data['sentence'], return_tensors='tf'))

        return inputs

    def train(self, training):
        for input, target in zip(*self.trainPreprocess(training)):

            berted = self.bert(input).last_hidden_state[0][1 : -1]

            if len(berted) > 0:
                self.model.fit(berted, target)

    def dev(self, deving):
        outputs = []
        for input, target in zip(*self.trainPreprocess(deving)):
            berted = self.bert(input).last_hidden_state[0][1 : -1]
            if len(berted) > 0:
                outputs.append(self.model.predict(berted))

        return outputs

    def predict(self, testing):
        outputs = []
        for input in self.testPreprocess(testing):
            berted = self.bert(input).last_hidden_state[0][1 : -1]
            if len(berted) > 0:
                outputs.append(self.model.predict(berted))
        return outputs

model = predictCloze(tokenizer, bert)

In [ ]:
model.train(training)

In [ ]:
model.model.save('./J/Models/1.0')

In [ ]:
tmp_model = predictCloze(tokenizer, bert)
tmp_model.model = tf.keras.models.load_model('./J/Models/1.0')

In [ ]:

with open("./CLOTH/CLOTH_test_cleaned.json", "r") as file:
    testing = json.load(file)
testing

In [ ]:
tmp_model.predict([{'sentence': replaced_sentence}])

In [ ]:
index =
print(testing[index]['sentence'])
replaced_sentence = testing[index]['sentence'].replace('[MASK]', testing[index]['answer'])
print(replaced_sentence)
predict = tf.math.top_k(tmp_model.predict([{'sentence': replaced_sentence}]), k=1, axis=-1)

print(predict)
tokenized = tokenizer(replaced_sentence, return_tensors='tf').input_ids[0][1:-1]

tokenizer.decode(tokenized[predict].numpy())

1/1 [==============================] - 0s 40ms/step


't h e'

In [ ]:
testing[0]['sentence']

"I remember sitting in that dark hall listening to Mr. Zigler  [MASK]  everyone's spirits up to the ceiling."

In [ ]:
deving

In [ ]:
dev = [dev[:, 0] for dev in devs]
dev

In [ ]:
index = 8

comp = list(zip(deving[0]['article'].split('.')[index].split(), dev[index]))
for token in comp:
    print(token)

# BERT + CRF


In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, TimeDistributed
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
input_ids = Input(shape=(64, ), name='input_ids', dtype='int32')
input_masks_ids = Input(shape=(64, ), name='attention_mask', dtype='int32')

x = bert.bert(input_ids, input_masks_ids).last_hidden_state
h = TimeDistributed(Dense(64, activation='softmax'))(x)
h = Dropout(0.5)(h)
decode_sequence, potentials, sequence_length, kernel = tfa.layers.CRF(5)(h)


In [ ]:
tmp = Model(inputs=[input_ids, input_masks_ids],
            outputs=[decode_sequence, potentials, sequence_length, kernel])
tmp.layers[2].trainable = False

In [ ]:
@tf.function
def crf_loss_func(potentials, sequence_length, kernel, y):
    crf_likelihood, transition_params = tfa.text.crf_log_likelihood(
        potentials, y, sequence_length, kernel
    )
    # likelihood to loss
    flat_crf_loss = -1 * crf_likelihood
    flat_crf_loss = flat_crf_loss
    crf_loss = tf.reduce_mean(flat_crf_loss)
    return crf_loss

optimizer = tf.keras.optimizers.Adam(0.02)
train_loss = tf.keras.metrics.Mean(name="train_loss")

@tf.function(experimental_relax_shapes=False)
def train_step(x, y):
    with tf.GradientTape() as tape:
        decoded_sequence, potentials, sequence_length, kernel = tmp(x)
        crf_loss = crf_loss_func(potentials, sequence_length, kernel, y)
        loss = crf_loss + tf.reduce_sum(tmp.losses)
    grads = tape.gradient(loss, tmp.trainable_variables)
    optimizer.apply_gradients(zip(grads, tmp.trainable_variables))

    train_loss(loss)

In [ ]:
epoch = 5
BATCH_SIZE = 2048

for i in range(epoch):
    train_loss.reset_states()
    for i in tqdm(range(0, len(targets), BATCH_SIZE)):
        upper = min(len(targets), i + BATCH_SIZE)
        try:
            with tf.device('/device:GPU:0'):
                train_step({k: v[i:upper] for k, v in inputs.items()}, targets[i:upper])
        except Exception as ERR:
            print(ERR)
            with tf.device('/CPU:0'):
                train_step({k: v[i:upper] for k, v in inputs.items()}, targets[i:upper])
    print(f"Epoch {i + 1}, " f"Loss: {train_loss.result()}")

100%|██████████| 77/77 [03:16<00:00,  2.55s/it]


Epoch 155649, Loss: 6.87814998626709


100%|██████████| 77/77 [03:00<00:00,  2.35s/it]


Epoch 155649, Loss: 6.53171443939209


100%|██████████| 77/77 [03:00<00:00,  2.35s/it]


Epoch 155649, Loss: 5.561308860778809


100%|██████████| 77/77 [03:00<00:00,  2.35s/it]


Epoch 155649, Loss: 4.394881248474121


100%|██████████| 77/77 [03:00<00:00,  2.34s/it]

Epoch 155649, Loss: 4.1543169021606445


In [ ]:
tmp.save('./J/Models/3.2')

In [ ]:
sentence = 'My grandfather has astonishing powers of recall . He can still vividly describe his first day at school as a child.'
clozed = tokenizer(sentence, truncation=True, padding='max_length', max_length=64, return_tensors='tf')


In [ ]:
pot, prob, *_ = tmp({k: v for k, v in clozed.items() if k != 'token_type_ids'})

In [ ]:
print(tf.math.top_k(prob[0, :, 1], k=20).indices)
print(tf.math.top_k(prob[0, :, 2], k=20).indices)
print(tf.math.top_k(prob[0, :, 3], k=20).indices)

tf.Tensor([12  7  4 14 13  5 11 20 62  8 47 61 42 32 60 59 33 49 31 46], shape=(20,), dtype=int32)
tf.Tensor([12  7  4 14 13  5 11 20 62  8 47 61 42 32 60 59 33 49 31 46], shape=(20,), dtype=int32)
tf.Tensor([12  7  4 14 13  5 11 20  8 62 61 47 32 42 60 59 33 49 31 44], shape=(20,), dtype=int32)


In [ ]:
index = tf.math.top_k(prob[0, :, 4], k=20).indices
index

<tf.Tensor: shape=(20,), dtype=int32, numpy=
array([12,  7,  4, 14, 13,  5, 11, 20, 62,  8, 47, 61, 42, 32, 60, 59, 33,
       49, 31, 46], dtype=int32)>

In [ ]:

print(tokenizer.decode(clozed.input_ids[0, 4:5]))

12
astonishing
7
astonishing
4
astonishing
14
astonishing
13
astonishing
5
astonishing
11
astonishing
20
astonishing
62
astonishing
8
astonishing
47
astonishing
61
astonishing
42
astonishing
32
astonishing
60
astonishing
59
astonishing
33
astonishing
49
astonishing
31
astonishing
46
astonishing


# TESTING

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, TimeDistributed
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
from tensorflow_addons.text.crf_wrapper import CRFModelWrapper

input_ids = Input(shape=(64, ), name='input_ids', dtype='int32')
input_masks_ids = Input(shape=(64, ), name='attention_mask', dtype='int32')

x = bert.bert(input_ids, input_masks_ids).last_hidden_state
h = TimeDistributed(Dense(64, activation='softmax'))(x)
y = Dropout(0.5)(h)


In [ ]:
tmp = Model(inputs=[input_ids, input_masks_ids],
            outputs=y)
tmp.layers[2].trainable = False


In [ ]:
tmpCRF = CRFModelWrapper(tmp, 5)
tmpCRF.compile(optimizer=Adam(2e-3))

In [ ]:
inputs['input_ids'].shape

(76850, 64)

In [ ]:
EPOCH = 1
BATCH_SIZE = 32
for _ in range(EPOCH):
    for i in tqdm(range(0, len(targets), BATCH_SIZE)):
        try:
            with tf.device('/device:GPU:0'):
                tmpCRF.train_step((
                    {k: v[i:min(i+BATCH_SIZE, len(targets))] for k, v in inputs.items()},
                            targets[i:min(i+BATCH_SIZE, len(targets))]))
        except Exception as ERR:
            print(ERR)
            with tf.device('/CPU:0'):
                tmpCRF.fit(
                          x = {k: v[i:min(i+BATCH_SIZE, len(targets))] for k, v in inputs.items()},
                           targets[i:min(i+BATCH_SIZE, len(targets))])
                          )

  0%|          | 0/2367 [00:00<?, ?it/s]

Exception encountered when calling layer 'crf_2' (type CRF).

Layer "dense_4" expects 1 input(s), but it received 4 input tensors. Inputs received: [<tf.Tensor: shape=(32, 64), dtype=int32, numpy=
array([[1, 4, 1, ..., 4, 1, 4],
       [1, 4, 1, ..., 4, 1, 4],
       [1, 4, 1, ..., 4, 1, 4],
       ...,
       [1, 4, 1, ..., 4, 1, 4],
       [1, 4, 1, ..., 4, 1, 4],
       [1, 4, 1, ..., 4, 1, 4]], dtype=int32)>, <tf.Tensor: shape=(32, 64, 5), dtype=float32, numpy=
array([[[ 2.86477376e-02,  7.32434988e-02,  2.12103594e-02,
          5.01345396e-02, -9.47735459e-03],
        [ 1.14318915e-02,  4.72668409e-02, -2.17819097e-03,
          3.09515372e-02, -7.83577748e-03],
        [ 1.90708283e-02,  2.07046838e-03, -1.67429037e-02,
          2.49647088e-02, -1.01003340e-02],
        ...,
        [-2.78591216e-02,  1.29941273e-02,  1.59486756e-03,
         -4.93555479e-02,  1.09166577e-02],
        [ 1.80469155e-02,  8.59981030e-02, -4.36769985e-03,
         -9.84838791e-03, -2.47488115e-02

  0%|          | 0/2367 [00:02<?, ?it/s]


ValueError: ignored

In [ ]:
tmpCRF({k: v[:1] for k, v in inputs.items()})

# 2.0


In [ ]:
from tensorflow.keras.models import load_model

tmp = load_model('./J/Models/2.0')

In [ ]:
index = 4
data = training_dataset[index]

answer = data['answer']
sentence = data['sentence']
replace_sentence = sentence.replace('[MASK]', answer)

tokenized = tokenizer(replace_sentence, return_tensors='tf')
tokenized

In [ ]:
seq, *_ = tmp({'input_token': tokenized['input_ids'],
     'masked_token': tokenized['attention_mask']})

tf.Tensor([[3 2 2 2 0 2 1 1 1 1 1 4 3 1 4 2 2 0 2]], shape=(1, 19), dtype=int32)
badly
By this time the car was missing so  [MASK]  it was shaking all over .This was the   worst  time to arrive at a garage--late Friday afternoon.
.


In [ ]:
np.where(seq==4)[1][0]

32

someone
The only reason was  that  they thought my name looked like the one of  [MASK]  who's on their wanted list and I had to wait till they checked me out  with  Washington.
[ C L S ]
o u t


In [ ]:
!pip install transformers
!pip install sentencepiece


In [ ]:
from transformers import BloomTokenizerFast, BloomForCausalLM

tokenizer = BloomTokenizerFast.from_pretrained('Langboat/bloom-6b4-zh')
model = BloomForCausalLM.from_pretrained('Langboat/bloom-6b4-zh')

In [ ]:
print(tokenizer.batch_decode(model.generate(tokenizer.encode('中国的首都是', return_tensors='pt'))))


NameError: ignored

In [ ]:
import sys
sys.path.insert(0, "/content/gdrive/Shareddrives/anzi")

In [ ]:
!pip install transformers

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393259 sha256=19633fd09d733fa73bceddd8ef0702b703e3d5a22139c8011cb6872efb62900f
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')  # start it
os.environ['DISPLAY']=':1.0'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 34 not upgraded.
Need to get 7,697 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libfontenc1 amd64 1:1.1.4-0ubuntu1 [14.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libxfont2 amd64 1:2.0.3-1 [91.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libxkbfile1 amd64 1:1.1.0-1 [65.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 x11-xkb-utils amd64 7.7+5 [158 kB]
Get:5 http://archive.ubuntu.com/ubunt

In [ ]:
import tkinter as tk
from anzi_v1_0 import SentenceandDistractors
import re

def process_input():
    sentences = Input.get("1.0","end-1c")

    for text in Texts:
        text.pack_forget()

    for sentence in sentences.split('\n'):
        if sentence != '':
            text = tk.Text(MiddleArea, height=5, width=100)
            data = SentenceandDistractors(sentence, v='3.2')
            Sentence = data['sentence'] + '\n(A) {} (B) {} (C) {} (D)'.format(data['distractor'][0], data['distractor'][1], data['distractor'][2], data['answer'])
            index = re.search('\[SEP\]', Sentence).start()
            Sentence = Sentence[6:index].replace('[MASK]', '____')
            text.insert(tk.END, )
            text.pack()
            Texts.append(text)


Texts = []

MAX_WIDTH = 700

Root = tk.Tk()

TopArea = tk.Frame(Root)
TopArea.pack()

MiddleArea = tk.Frame(Root)
MiddleArea.pack(pady=10)

Input = tk.Text(TopArea, height=10, width=100)
Input.pack(side='left', pady=20, padx=20)

Button = tk.Button(TopArea, text='Process', command=process_input)
Button.pack(side='left', padx=20)





Root.mainloop()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


OSError: ignored

# T5

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.9 MB/s eta 0:00:00


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("Langboat/mengzi-t5-base-mt")
model = T5ForConditionalGeneration.from_pretrained("Langboat/mengzi-t5-base-mt")


ImportError: ignored